In [6]:
import numpy             as np
import matplotlib.pyplot as plt
import scipy.sparse      as sparse
import scipy.special     as sp

import hylife.utilitis_FEEC.bsplines             as bsp
import hylife.utilitis_FEEC.projectors_global    as proj_glob
import hylife.utilitis_FEEC.projectors_local     as proj_loc
import hylife.utilitis_FEEC.derivatives          as der
import hylife.utilitis_FEEC.evaluation           as eva
import hylife.geometry.mappings_analytical       as mapping
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D

In [7]:
Nel       = [16, 4, 4]              # mesh generation on logical domain
bc        = [True, True, True]      # boundary conditions (True: periodic, False: else)
p         = [3, 2, 2]               # spline degrees  


el_b      = [np.linspace(0., 1., Nel + 1) for Nel in Nel]                      # element boundaries
delta     = [1/Nel for Nel in Nel]                                             # element sizes
T         = [bsp.make_knots(el_b, p, bc) for el_b, p, bc in zip(el_b, p, bc)]  # knot vectors (for N functions)
t         = [T[1:-1] for T in T]                                               # reduced knot vectors (for D function)
NbaseN    = [Nel + p - bc*p for Nel, p, bc in zip(Nel, p, bc)]                 # number of basis functions (N functions)
NbaseD    = [NbaseN - (1 - bc) for NbaseN, bc in zip(NbaseN, bc)]              #number of basis functions (D functions)

kind_map = 1               #slab geometry
L_map    = [3., 2., 1.] #length of the simulation box 

args_map = kind_map, L_map #putting it together
rho_ini_phys = lambda x, y, z : np.sin(2*np.pi*x)

In [8]:
vx_ini_phys = lambda x, y, z : 1.
vy_ini_phys = lambda x, y, z : 0.
vz_ini_phys = lambda x, y, z : 0.

In [9]:
def rho_ini(xi1_arr, xi2_arr, xi3_arr):
    n1, n2, n3 = xi1_arr.shape #xi1_arr Logical Coordinates (?)
    rho = np.empty((n1, n2, n3), dtype=float) # Returns numpy array of shape [n1,n2,n3] same fct as np.zeros
    for i1 in range(n1):
        for i2 in range(n2):
            for i3 in range(n3):
                idx = i1, i2, i3
                xi1 = xi1_arr[idx]
                xi2 = xi2_arr[idx]
                xi3 = xi3_arr[idx]
                x = mapping.f(xi1, xi2, xi3, *args_map, 1) #Analytic Mapping to slab Geometry
                y = mapping.f(xi1, xi2, xi3, *args_map, 2)
                z = mapping.f(xi1, xi2, xi3, *args_map, 3)
                rho[idx] = rho_ini_phys(x, y, z) * mapping.det_df(xi1, xi2, xi3, *args_map)
    return rho

In [10]:
def v1_ini(xi1, xi2_arr, xi3_arr):
    n2, n3 = xi2_arr.shape
    v1 = np.empty((n2, n3), dtype=float)
    for i2 in range(n2):
        for i3 in range(n3):
            idx = i2, i3
            
            xi2 = xi2_arr[idx]
            xi3 = xi3_arr[idx]

            x = mapping.f(xi1, xi2, xi3, *args_map, 1)
            y = mapping.f(xi1, xi2, xi3, *args_map, 2)
            z = mapping.f(xi1, xi2, xi3, *args_map, 3)

            df_11 = mapping.df(xi1, xi2, xi3, *args_map, 11)
            df_21 = mapping.df(xi1, xi2, xi3, *args_map, 21)
            df_31 = mapping.df(xi1, xi2, xi3, *args_map, 31)

            v1[idx] = vx_ini_phys(x, y, z) * df_11 + vy_ini_phys(x, y, z) * df_21 + vz_ini_phys(x, y, z) * df_31
    return v1

In [32]:
def v1_pr(xi1_arr, xi2_arr, xi3_arr):
    n1= xi1_arr.shape[0] #xi1_arr Logical Coordinates (?)
    n2= xi2_arr
    n3= xi3_arr.shape
    print(n2)
    v1 = [] # Returns numpy array of shape [n1,n2,n3] same fct as np.zeros
    for i1 in range(n1):
        for i2 in range(n2):
            for i3 in range(n3):
                idx = i1, i2, i3
                xi1 = xi1_arr[idx]
                xi2 = xi2_arr[idx]
                xi3 = xi3_arr[idx]
                x = mapping.f(xi1, xi2, xi3, *args_map, 1) #Analytic Mapping to slab Geometry
                y = mapping.f(xi1, xi2, xi3, *args_map, 2)
                z = mapping.f(xi1, xi2, xi3, *args_map, 3)
                v1.append(1)
    return v1

In [12]:
def v1_times_rho(xi1, xi2_arr, xi3_arr):
    global rho_coeff
    n2, n3 = xi2_arr.shape
    fun = np.empty((n2, n3), dtype=float)
    
    xi2_arr1d = xi2_arr[:, 0]
    xi3_arr1d = xi3_arr[0, :]
    rho_h = eva.FEM_field_V3_3d(rho_coeff, [np.array([xi1]), xi2_arr1d, xi3_arr1d], T, p, bc)

    fun = rho_h.reshape(n2, n3) * v1_ini(xi1, xi2_arr, xi3_arr)
    
    return fun

In [13]:
def v2_times_rho(xi1_arr, xi2, xi3_arr):
    global rho_coeff
    #global v_coeff  # [v1_coeff,v2_coeff,v3_coeff]
    n1, n3 = xi1_arr.shape
    fun = np.empty((n1, n3), dtype=float)
    
    xi1_arr1d = xi1_arr[:, 0]
    xi3_arr1d = xi3_arr[0, :]
    rho_h = eva.FEM_field_V3_3d(rho_coeff, [xi1_arr1d, np.array([xi2]), xi3_arr1d], T, p, bc)
    #[v1_h, v2_h, v3_h] = eva.FEM_field_V1_3d(v_coeff, [xi1_arr1d, np.array([xi2]), xi3_arr1d], T, p, bc)
    fun = rho_h.reshape(n1, n3)*0. #* v2_h.reshape(n1, n3)
    return fun

In [14]:
def v3_times_rho(xi1_arr, xi2_arr, xi3):
    global rho_coeff
    n1, n2 = xi1_arr.shape
    fun = np.empty((n1, n2), dtype=float)
    
    xi1_arr1d = xi1_arr[:, 0] #Take All the diferent coordinates in x1 as a 1d_array
    xi2_arr1d = xi2_arr[0, :] #Take All the diferent coordinates in x2 as a 1d_array
    # Third coordinate xi3 is the one that the function (rho*v)3 depends on
    rho_h = eva.FEM_field_V3_3d(rho_coeff, [xi1_arr1d, xi2_arr1d, np.array([xi3])], T, p, bc)   
    fun = rho_h.reshape(n1, n2) * 0.
    
    return fun

In [15]:
proj_glob = proj_glob.projectors_3d(T, p, bc)
proj_loc  = proj_loc.projectors_local_3d(T, p, bc)

In [16]:
proj_glob.assemble_V3()
proj_glob.assemble_V2()
proj_glob.assemble_V1()
derivatives = der.discrete_derivatives(T, p, bc)
DIV  = derivatives.DIV_3d()

#### $\rho_0$ Initialization and Visualization

In [17]:
rho_coeff = proj_glob.PI_3(rho_ini)

In [33]:
v_coeff= proj_glob.PI_1([v1_pr,v1_pr,v1_pr])

0.125


TypeError: 'numpy.float64' object cannot be interpreted as an integer

In [ ]:
%matplotlib notebook
k0=p[0]
k1=p[1]
k2=p[2]
x=np.linspace(0,1,100)
y=np.linspace(0,1,100)
z=np.array([0])
rho_0_plot=eva.FEM_field_V3_3d(rho_coeff, [x,y,z], T, p, bc)
rho_0_plot=rho_0_plot.reshape(len(x),len(y))
x_m, y_m=np.meshgrid(x,y)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(x_m,y_m,rho_0_plot)

In [ ]:
def eva_rho_t():
    global rho_coeff
    rhov1_coeff, rhov2_coeff, rhov3_coeff = proj_glob.PI_2([v1_times_rho, v2_times_rho, v3_times_rho])
    rho_t = -(DIV.dot(np.concatenate((rhov1_coeff.flatten(), rhov2_coeff.flatten(), rhov3_coeff.flatten())))).reshape(NbaseD[0], NbaseD[1], NbaseD[2]) 
    return rho_t

In [ ]:
#Explicit Euler Integration
dt=0.1
rho_t=eva_rho_t()
rho_coeff=np.copy(rho_coeff+dt*rho_t)

In [ ]:
rho_0_plot=eva.FEM_field_V3_3d(rho_coeff_1, [x,y,z], T, p, bc)
rho_0_plot=rho_0_plot.reshape(len(x),len(y))
x_m, y_m=np.meshgrid(x,y)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(x_m,y_m,rho_0_plot)